In [ ]:
from torch import nn
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]]).to(device)
y_train = torch.tensor([0., 1., 1., 0.]).reshape((4, 1)).to(device)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(in_features=2, out_features=3),
            nn.ReLU(),
            nn.Linear(in_features=3, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.seq(x)


def train(m: NeuralNetwork, x, y, epochs=10000):
    optimizer = torch.optim.SGD(m.parameters(), lr=0.1)
    for i in range(epochs):
        y_pred = model(x_train)

        loss = torch.nn.BCELoss()(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model = NeuralNetwork().to(device)
train(model, x_train, y_train)

In [ ]:
print('loss:', torch.nn.BCELoss()(model(x_train), y_train).item())
print('res:', model(x_train).detach().cpu().numpy())

In [ ]:
import numpy as np
from sklearn.datasets import load_digits

def as_categorical(x):
    t = np.zeros((10, 1))
    t[x] = 1
    return t

model = nn.Sequential(
    nn.Linear(in_features=8 * 8, out_features=20),
    nn.ReLU(),
    nn.Linear(in_features=20, out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10, out_features=10),
    nn.Softmax(dim=1)
)

digits = load_digits()
n = len(digits.images)
x_all = torch.from_numpy(digits.images).reshape(n, -1).to(torch.float) / 255
y_all = torch.from_numpy(np.asarray([as_categorical(target) for target in digits.target])).reshape(n, 10).to(torch.float)
TRAIN_CNT = 1000
EVAL_CNT = n - TRAIN_CNT
x_train = x_all[:TRAIN_CNT]
x_test = x_all[TRAIN_CNT:TRAIN_CNT + EVAL_CNT]

y_train = y_all[:TRAIN_CNT]
y_test = y_all[TRAIN_CNT:TRAIN_CNT + EVAL_CNT]

def train(m: NeuralNetwork, x, y, epochs=10000):
    optimizer = torch.optim.Adam(m.parameters())
    for i in range(epochs):
        y_pred = model(x)
        loss = torch.nn.CrossEntropyLoss()(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print(loss.item())

train(model, x_train, y_train, epochs=10000)

In [ ]:
with torch.no_grad():
    y_pred = model(x_test)
    y_pred = torch.argmax(y_pred, dim=1)
    y_real = torch.argmax(y_test, dim=1)
    cor = (y_pred == y_real).sum().item()
    wa = len(y_real) - cor
    print(f'accuracy: {cor / y_real.shape[0]}')